##### Here I am implementing Decision Tree classifier from scratch. There are several algorithms to make decision trees but out of them 2 are used mostly named: ID3(Iterative Dichotomiser 3) and CART(Classification And Regression Trees). In this notebook I will be implementing a decision tree classifier using ID3 algorithm.

##### Some main terms included in ID3 algorihtm are : impurity, entropy & information gain.

In [1]:
# imports ;
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

##### I am using OO methodology for implementing this algorithm with several helper functions.

In [6]:
class ID3Classifier:
    # calculating entropy:
    def entropy(self,feature):
        # finding unique values and feature values of a feature
        values,counts=np.unique(feature,return_counts=True)
        entropy_list=[]
        for i in range(len(values)):
            prob=counts[i]/np.sum(counts)
            entropy_list.append(-prob*np.log2(prob))
            total_entropy=np.sum(entropy_list)
            return total_entropy

    # calculating information gain
    def info_gain(self,data,feature_name,target_name):
        total_entropy=self.entropy(data[target_name])
        values,counts=np.unique(data[feature_name],return_counts=True)
        weighted_entropy_list=[]
        for i in range(len(values)):
            sub_prob=counts[i]/np.sum(counts)
            sub_entropy=self.entropy(data.where(data[feature_name]==values[i]).dropna()[target_name])
            weighted_entropy_list.append(sub_prob*sub_entropy)
        total_weighted_entropy=np.sum(weighted_entropy_list)
        information_gain=total_entropy-total_weighted_entropy
        return information_gain
    
    # Making the decision tree:
    def make_decision_tree(self,data,original_data,feature_name,target_name,parent_node=None):
        # base case: if data is pure return majority class of subset
        unique_class=np.unique(data[target_name])
        if len(unique_class)<=1:
            return unique_class[0]
        # if subset is empty or no samples are there return majority class
        elif len(data)==0:
            majority_class_index=np.argmax(np.unique(original_data[target_name],return_counts=True)[1])
            return np.unique(original_data[target_name])[majority_class_index]
        # if dataset has no features return parent node
        elif len(feature_name)==0:
            return parent_node
        # if none of the conditions is true construct a new branch
        else:
            # determine parent node class of current branch
            majority_class_index = np.argmax(np.unique(data[target_name], return_counts=True)[1])
            parent_node = unique_class[majority_class_index]
            # find information gain and choose feature which best splits the data
            info_gain_values=[self.info_gain(data,feature,target_name) for feature in feature_name]
            best_feature_index=np.argmax(info_gain_values)
            best_feature=feature_name[best_feature_index]
            # create tree first emmpty
            tree={best_feature:{}}
            # best feature becomes parent node so removinf it from features:
            feature_name=[i for i in feature_name if i!=best_feature]
            # creating other nodes
            parent_values=np.unique(data[best_feature])
            for value in parent_values:
                sub_data=data.where(data[best_feature]==value).dropna()
                # recursively calling the same algorithm
                subtree=self.make_decision_tree(sub_data,original_data,feature_name,target_name,parent_node)
                # add subtree to original tree
                tree[best_feature][value]=subtree
            return tree

    # defining the make prediction function while traversing the tree
    def make_prediction(self,sample,tree,default=1):
        for attribute in list(sample.keys()):
            if attribute in list(tree.keys()):
                try:
                    result=tree[attribute][sample[attribute]]
                except:
                    return default
                result=tree[attribute][sample[attribute]]
                # if there are more attributes recursively find best result
                if isinstance(result,dict):
                    return self.make_prediction(sample,result)
                else:
                    return result

    # defining the predict function:
    def predict(self, input):
    # convert input data into a dictionary of samples
        samples = input.to_dict(orient='records')
        predictions = []
        # make a prediction for every sample
        for sample in samples:
            predictions.append(self.make_prediction(sample, self.tree, 1.0))
        return predictions

    # make the last fit function:
    def fit(self, input, output):
        data = input.copy()
        data[output.name] = output
        self.tree = self.make_decision_tree(data, data, input.columns, output.name)


##### Testing the above built tree with some input data:

In [8]:
data_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data"
df = pd.read_csv(data_url, header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0


In [9]:
# rename known columns
columns = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg',
           'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'disease_present']
df.columns = columns

In [10]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,disease_present
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0


In [11]:
df.disease_present.value_counts()

0    164
1     55
2     36
3     35
4     13
Name: disease_present, dtype: int64

In [12]:
# convert disease_present feature to binary
df['disease_present'] = df.disease_present.replace([1,2,3,4], 1)
# drop rows with missing values, missing = ?
df = df.replace("?", np.nan)
df = df.dropna()

In [16]:
# organize data into input and output
X = df.drop(columns="disease_present")
y = df["disease_present"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.23)

In [17]:
# initialize and fit model
model = ID3Classifier()
model.fit(X_train, y_train)

In [18]:
# return accuracy score
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

0.4927536231884058

##### The accuracy obtained is very less as I implemented a very simple tree but my main intention was to understand whats happening under the hood in decision tree classifier algorithm. Its a sign of underfitting.